In [2]:
import networkx as nx
import numpy as np
import plotly.graph_objects as go
import matplotlib as mpl
import pandas as pd
from IPython.display import clear_output
from plotly.subplots import make_subplots
from matplotlib import pyplot as plt
from qiskit import Aer
from qiskit import QuantumCircuit
from qiskit.visualization import plot_state_city
from qiskit.algorithms.optimizers import COBYLA, SLSQP, ADAM
from time import time
from copy import copy
from typing import List
from qc_grader.graph_util import display_maxcut_widget, QAOA_widget, graphs
from qiskit_optimization import QuadraticProgram
mpl.rcParams['figure.dpi'] = 300

# Loading your IBM Quantum account(s)
#provider = IBMQ.load_account()

In [3]:
graph = nx.Graph()
#Add nodes and edges
graph.add_nodes_from(np.arange(0,6,1))


LRTs = [52,4,80,103,17,32,140,26,77,8,91,84,60]



edges = [(0,1,52.0),(0,2,4.0),(0,3,80.0),(0,4,103.0),(0,5,17.0),(1,2,32.0),(1,3,140.0),(1,4,26.0),
         (1,5,77.0),(2,4,8.0),(2,5,91.0),(3,4,84.0),(3,5,60.0)]
graph.add_weighted_edges_from(edges)
graphs['custom'] = graph
#Display widget
display_maxcut_widget(graphs['custom'])

CytoscapeWidget(cytoscape_layout={'name': 'cola'}, cytoscape_style=[{'selector': 'node', 'css': {'background-c…

Output()

In [4]:
def maxcut_cost_fn(graph: nx.Graph, bitstring: List[int]) -> float:
    """
    Computes the maxcut cost function value for a given graph and cut represented by some bitstring
    Args:
        graph: The graph to compute cut values for
        bitstring: A list of integer values '0' or '1' specifying a cut of the graph
    Returns:
        The value of the cut
    """
    #Get the weight matrix of the graph
    weight_matrix = nx.adjacency_matrix(graph).toarray()
    size = weight_matrix.shape[0]
    value = 0.

    #INSERT YOUR CODE TO COMPUTE THE CUT VALUE HERE
    for m in range(size): 
        for n in range(size):
            value += weight_matrix[m,n] * bitstring[m] * (1-bitstring[n])

    return value

def plot_maxcut_histogram(graph: nx.Graph) -> None:
    """
    Plots a bar diagram with the values for all possible cuts of a given graph.
    Args:
        graph: The graph to compute cut values for
    """
    num_vars = graph.number_of_nodes()
    #Create list of bitstrings and corresponding cut values
    bitstrings = ['{:b}'.format(i).rjust(num_vars, '0')[::-1] for i in range(2**num_vars)]
    values = [maxcut_cost_fn(graph = graph, bitstring = [int(x) for x in bitstring]) for bitstring in bitstrings]
    #Sort both lists by largest cut value
    values, bitstrings = zip(*sorted(zip(values, bitstrings)))
    #Plot bar diagram
    bar_plot = go.Bar(x = bitstrings, y = values, marker=dict(color=values, colorscale = 'plasma', colorbar=dict(title='Cut Value')))
    fig = go.Figure(data=bar_plot, layout = dict(xaxis=dict(type = 'category'), width = 1500, height = 600))
    fig.show()


In [18]:
plot_maxcut_histogram(graph = graphs['custom'])

Traceback (most recent call last):
  File "<ipython-input-18-9bd2fdc5e55e>", line 1, in <module>
    plot_maxcut_histogram(graph = graphs['custom'])
  File "<ipython-input-17-42f1fa5e66c0>", line 31, in plot_maxcut_histogram
    values = [maxcut_cost_fn(graph = graph, bitstring = [int(x) for x in bitstring]) for bitstring in bitstrings]
  File "<ipython-input-17-42f1fa5e66c0>", line 31, in <listcomp>
    values = [maxcut_cost_fn(graph = graph, bitstring = [int(x) for x in bitstring]) for bitstring in bitstrings]
  File "<ipython-input-17-42f1fa5e66c0>", line 16, in maxcut_cost_fn
    for m in range(bitstring):
TypeError: 'list' object cannot be interpreted as an integer

Use %tb to get the full traceback.


In [ ]:
def quadratic_program_from_graph(graph: nx.Graph) -> QuadraticProgram:
    """Constructs a quadratic program from a given graph for a MaxCut problem instance.
    Args:
        graph: Underlying graph of the problem.
    Returns:
        QuadraticProgram
    """
    #Get weight matrix of graph
    weight_matrix = nx.adjacency_matrix(graph)
    shape = weight_matrix.shape
    size = shape[0]
    #Build qubo matrix Q from weight matrix W
    qubo_matrix = np.zeros((size, size))
    qubo_vector = np.zeros(size)
    for i in range(size):
        for j in range(size):
            qubo_matrix[i, j] -= weight_matrix[i, j]
    for i in range(size):
        for j in range(size):
            qubo_vector[i] += weight_matrix[i,j]

    #INSERT YOUR CODE HERE
    quadratic_program = QuadraticProgram('MAXCUT')
    
    quadratic_program.binary_var(name = 'x_0')
    quadratic_program.binary_var(name = 'x_1')
    quadratic_program.binary_var(name = 'x_2')
    quadratic_program.binary_var(name = 'x_3')
    quadratic_program.binary_var(name = 'x_4')
    quadratic_program.binary_var(name = 'W_0')
    quadratic_program.binary_var(name = 'W_1')
    quadratic_program.binary_var(name = 'W_2')
    quadratic_program.binary_var(name = 'W_3')
    quadratic_program.binary_var(name = 'W_4')
    
    
    quadratic = qubo_matrix
    linear = qubo_vector
    quadratic_program.maximize(quadratic = qubo_matrix, linear = qubo_vector)
    
    return quadratic_program